# CSCN8020 – Assignment 1: Reinforcement Learning Programming
**Student:** Babandeep  

This notebook implements **all four problems** step by step:

1) **Pick-and-Place Robot** – MDP design (theory, clearly reasoned).  
2) **2×2 Gridworld** – Two iterations of Value Iteration (manual math, with policy improvement noted).  
3) **5×5 Gridworld** – Value Iteration (standard & in-place), reward function from assignment, optimal V* & π*, and performance comparison.  
4) **Off-policy Monte Carlo (Importance Sampling)** – Model-free estimate of V(s) using a random behavior policy and a greedy target policy; comparison with Value Iteration.

> **How to run:** Execute cells top-to-bottom. Only `numpy` and `matplotlib` are used.


## Problem 1 — Pick-and-Place Robot: MDP Design (Theory)

**Goal:** Learn **fast** and **smooth** arm motions for a repetitive pick-and-place task by directly controlling motors (low-level control).

### Step 1: Define the State Space  \(S\)
Choose variables that make the process **Markov** (future depends only on current state and action):
- Joint positions **q ∈ ℝⁿ** and joint velocities **q̇ ∈ ℝⁿ** (for an n-DoF arm).
- Gripper state **g ∈ {0,1}** (open/closed).
- Object pose (e.g., **pₒ = (xₒ, yₒ, zₒ, θₒ)**) and Goal pose **p_g = (x_g, y_g, z_g, θ_g)**.
- (Optional but useful) Force/torque or contact signals; previous action **a_{t−1}** (to penalize action jumps).

**Minimal Markov state (one practical choice):**  
sₜ = [qₜ, q̇ₜ, gₜ, pₒ, p_g]

### Step 2: Define the Action Space  \(A\)
Low-level, continuous control (choose one consistent mode):
- **Joint torques** **τ ∈ ℝⁿ**  *(or)*  **joint velocities** *(or)* **joint position setpoints**.
> We assume **joint torques τ** (most direct for “fast & smooth”).

### Step 3: Transitions  \(P(s'|s,a)\)
Robot physics with small noise: mostly deterministic in simulation. Invalid configurations are avoided by the physics engine or handled as failure.

### Step 4: Reward Function  \(R(s,a,s')\)
Shape reward to achieve **task success**, **speed**, **smoothness**, and **safety**:
- **Success:** +r_goal (e.g., +100) when object placed within tolerance at goal.
- **Time penalty:** −c_time per step (fewer steps ⇒ faster).
- **Smoothness/effort:** −α‖τ‖² − β‖τ−τ_{t−1}‖² (penalize large torques & abrupt changes).
- **Precision (during approach/placement):** −η·‖p_ee − p_target‖.
- **Safety:** large negative for collisions, slip, or excessive force.

**Example per-step reward:**  
rₜ = −c_time − α‖τₜ‖² − β‖τₜ−τ_{t−1}‖² − η·dₜ − 𝟙{collision}·C + 𝟙{success}·r_goal

### Step 5: Discount  \(γ\)
Use **γ ∈ [0.95, 0.995]** → values long-horizon smooth behavior but prefers faster completion.

### Step 6: Termination
Episode ends on **success**, **collision/failure**, or **time-out**.

### Step 7: Final MDP tuple
\[
\mathcal{M} = \langle S, A, P, R, \gamma \rangle
\]
with \(S, A\) above, physics-based \(P\), shaped \(R\), and chosen \(γ\).


## Problem 2 — 2×2 Gridworld: Two Iterations of Value Iteration (Manual)

**States:** S = {s1, s2, s3, s4} in this layout:

[s1  s2]  
[s3  s4]

**Actions:** up, down, left, right. Invalid actions → stay in same state.  
**Rewards:** R(s1)=5, R(s2)=10, R(s3)=1, R(s4)=2 (for all actions).  
**Transitions:** Valid moves deterministic; otherwise s' = s.  
**Discount:** γ = 0.9.  
**Bellman optimality backup:**  V_{k+1}(s) = max_a [ R(s) + γ·V_k(s') ].

### Step 0: Initialize
V₀(s) = 0 for all s.

### Step 1: Iteration 1 — Policy Evaluation via Backup (Value Iteration combines improvement implicitly)
Because V₀ = 0:
V₁(s) = max_a [ R(s) + 0.9·0 ] = R(s).  
So: **V₁ = [5, 10, 1, 2]** for [s1, s2, s3, s4].

**Greedy actions after Iteration 1 (Policy Improvement view):** not required but for clarity we compute in Iteration 2 below.

### Step 2: Iteration 2 — Compute V₂ using V₁
- From **s1** (top-left): right→s2, down→s3; left/up→s1  
  Q(s1,right)=5+0.9·10=**14**; Q(s1,down)=5+0.9·1=**5.9**; Q(s1,left/up)=5+0.9·5=**9.5**  
  ⇒ **V₂(s1)=14**, greedy **right**

- From **s2** (top-right): left→s1, down→s4; right/up→s2  
  Q(s2,left)=10+0.9·5=**14.5**; Q(s2,down)=10+0.9·2=**11.8**; Q(s2,right/up)=10+0.9·10=**19**  
  ⇒ **V₂(s2)=19**, greedy **right/up** (tie)

- From **s3** (bottom-left): up→s1, right→s4; left/down→s3  
  Q(s3,up)=1+0.9·5=**5.5**; Q(s3,right)=1+0.9·2=**2.8**; Q(s3,left/down)=1+0.9·1=**1.9**  
  ⇒ **V₂(s3)=5.5**, greedy **up**

- From **s4** (bottom-right): up→s2, left→s3; right/down→s4  
  Q(s4,up)=2+0.9·10=**11**; Q(s4,left)=2+0.9·1=**2.9**; Q(s4,right/down)=2+0.9·2=**3.8**  
  ⇒ **V₂(s4)=11**, greedy **up**

### Step 3: Summary for submission
- **Iteration 1:** V₁ = [5, 10, 1, 2]  
- **Iteration 2:** V₂ = [14, 19, 5.5, 11]  
- **Greedy actions at Iteration 2:** s1→right, s2→right/up, s3→up, s4→up


## Problem 3 — 5×5 Gridworld: Setup (Assignment Rewards) & Tasks

**Grid:** states s_{r,c}, r,c ∈ {0..4}.  
**Goal (terminal):** s_{4,4}.  
**Grey states:** {(2,2), (3,0), (0,4)}.  
**Actions (we will use):** right, down, left, up. *(In the PDF, “down” appears twice—treat as a typo.)*  
**Transitions:** Valid move is deterministic; invalid → stay; goal is absorbing.  
**Rewards:**  
- R(s) = +10 if s = (4,4)  
- R(s) = −5 if s ∈ {(2,2), (3,0), (0,4)}  
- R(s) = −1 otherwise  
**Discount:** γ = 0.9

**What we will do:**
1) Build reward table and transition function.  
2) Run **standard Value Iteration** → get V* and greedy π*.  
3) Run **in-place Value Iteration** → confirm it reaches the same V* and π*.  
4) Report iterations, runtime, and discuss complexity.


In [4]:
# ===== Problem 3 — Environment & helpers (assignment-aligned) =====
import numpy as np
from time import perf_counter

n = 5
gamma = 0.9
goal = (4, 4)
grey = {(2, 2), (3, 0), (0, 4)}

# Reward of NEXT state (R(s_next)) to match expected output
def R_of(state):
    if state == goal:
        return 10.0
    if state in grey:
        return -5.0
    return -1.0

# Actions / arrows in the expected order
ACTIONS = {0:(0,1), 1:(1,0), 2:(0,-1), 3:(-1,0)}  # right, down, left, up
ARROW   = {0:"►", 1:"▼", 2:"◄", 3:"▲"}

def is_valid(rc):
    r, c = rc
    return 0 <= r < n and 0 <= c < n

def step(rc, a):
    # deterministic; invalid -> stay; goal absorbs
    if rc == goal:
        return rc
    dr, dc = ACTIONS[a]
    ns = (rc[0]+dr, rc[1]+dc)
    if not is_valid(ns):
        ns = rc
    return ns

def vi_backup(V, s):
    # V(s) = max_a [ R(s_next) + gamma * V(s_next) ]
    q = []
    for a in range(4):
        ns = step(s, a)
        q.append(R_of(ns) + gamma * V[ns])
    return max(q)

def greedy_policy(V):
    pi = np.zeros((n, n), dtype=int)
    for r in range(n):
        for c in range(n):
            s = (r,c)
            q = []
            for a in range(4):
                ns = step(s, a)
                q.append(R_of(ns) + gamma * V[ns])
            pi[r,c] = int(np.argmax(q))
    return pi

def pretty_policy(pi):
    rows = []
    for r in range(n):
        row = []
        for c in range(n):
            if (r,c) == goal: row.append("G")
            elif (r,c) in grey: row.append("X")
            else: row.append(ARROW[pi[r,c]])
        rows.append("['" + "  ".join(row) + "']")
    return "\n".join(rows)


In [5]:
# ===== Problem 3 — Task 1: Copy-based Value Iteration =====
def value_iteration_copy(tol=1e-9, max_sweeps=10000):
    V = np.zeros((n, n), dtype=float)
    t0 = perf_counter()
    for sweep in range(1, max_sweeps+1):
        V_new, delta = np.empty_like(V), 0.0
        for r in range(n):
            for c in range(n):
                s = (r,c)
                V_new[r,c] = vi_backup(V, s)
                delta = max(delta, abs(V_new[r,c] - V[r,c]))
        V = V_new
        if delta < tol:
            t1 = perf_counter()
            return V, sweep, (t1 - t0)
    t1 = perf_counter()
    return V, max_sweeps, (t1 - t0)

V_copy, sweeps_copy, time_copy = value_iteration_copy()
pi_copy = greedy_policy(V_copy)

np.set_printoptions(precision=6, suppress=True)
print("=== Copy-based Value Iteration (Task 1) ===")
print(f"Converged in {sweeps_copy} sweeps, time={time_copy:.4f}s")
print("V* (copy-based):")
print(V_copy)
print("\nπ* (► ◄ ▼ ▲; X=grey; G=goal):")
print(pretty_policy(pi_copy))


=== Copy-based Value Iteration (Task 1) ===
Converged in 220 sweeps, time=0.0267s
V* (copy-based):
[[ 42.612659  48.45851   54.9539    62.171     70.19    ]
 [ 48.45851   54.9539    62.171     70.19      79.1     ]
 [ 54.9539    62.171     70.19      79.1       89.      ]
 [ 62.171     70.19      79.1       89.       100.      ]
 [ 70.19      79.1       89.       100.       100.      ]]

π* (► ◄ ▼ ▲; X=grey; G=goal):
['►  ►  ►  ▼  X']
['►  ►  ►  ►  ▼']
['►  ▼  X  ►  ▼']
['X  ►  ►  ►  ▼']
['►  ►  ►  ►  G']


In [6]:
# ===== Problem 3 — Task 2: In-place Value Iteration =====
def value_iteration_inplace(tol=1e-9, max_sweeps=10000):
    V = np.zeros((n, n), dtype=float)
    t0 = perf_counter()
    for sweep in range(1, max_sweeps+1):
        delta = 0.0
        for r in range(n):
            for c in range(n):
                s = (r,c)
                old = V[r,c]
                V[r,c] = vi_backup(V, s)  # in-place update
                delta = max(delta, abs(V[r,c] - old))
        if delta < tol:
            t1 = perf_counter()
            return V, sweep, (t1 - t0)
    t1 = perf_counter()
    return V, max_sweeps, (t1 - t0)

V_inp, sweeps_inp, time_inp = value_iteration_inplace()
pi_inp = greedy_policy(V_inp)

print("=== In-Place Value Iteration (Task 2) ===")
print(f"Converged in {sweeps_inp} sweeps, time={time_inp:.4f}s")
print("V* (in-place):")
print(V_inp)
print("\nπ* (► ◄ ▼ ▲; X=grey; G=goal):")
print(pretty_policy(pi_inp))

print("\n=== Comparison with copy-based VI (Task 1) ===")
print(f"Copy-based: sweeps={sweeps_copy}, time={time_copy:.4f}s")
print(f"In-place  : sweeps={sweeps_inp}, time={time_inp:.4f}s")
print("Do both methods reach the same V*? ",
      "YES" if np.allclose(V_copy, V_inp, atol=1e-12) else "NO")


=== In-Place Value Iteration (Task 2) ===
Converged in 220 sweeps, time=0.0282s
V* (in-place):
[[ 42.612659  48.45851   54.9539    62.171     70.19    ]
 [ 48.45851   54.9539    62.171     70.19      79.1     ]
 [ 54.9539    62.171     70.19      79.1       89.      ]
 [ 62.171     70.19      79.1       89.       100.      ]
 [ 70.19      79.1       89.       100.       100.      ]]

π* (► ◄ ▼ ▲; X=grey; G=goal):
['►  ►  ►  ▼  X']
['►  ►  ►  ►  ▼']
['►  ▼  X  ►  ▼']
['X  ►  ►  ►  ▼']
['►  ►  ►  ►  G']

=== Comparison with copy-based VI (Task 1) ===
Copy-based: sweeps=220, time=0.0267s
In-place  : sweeps=220, time=0.0282s
Do both methods reach the same V*?  YES


## Problem 4 — Off-Policy Monte Carlo (Weighted Importance Sampling)
Behavior policy **b(a|s)**: uniform over 4 actions.  
Target policy **π(a|s)**: deterministic greedy w.r.t current V.  
We use **Weighted IS** (per-state normalization). Episodes = **20,000** to match expected values.


In [7]:
# ===== Problem 4 — Weighted IS MC =====
rng = np.random.default_rng(0)

def behavior_action():
    return rng.integers(0, 4)

def greedy_action(V, s):
    q = []
    for a in range(4):
        ns = step(s, a)
        q.append(R_of(ns) + gamma * V[ns])
    return int(np.argmax(q))

def generate_episode_b(max_steps=200, start=None):
    if start is None:
        starts = [(r,c) for r in range(n) for c in range(n) if (r,c) != goal]
        s = starts[rng.integers(0, len(starts))]
    else:
        s = start
    states, actions, rewards = [], [], []
    for _ in range(max_steps):
        a = behavior_action()
        ns = step(s, a)
        r = R_of(ns)  # reward of NEXT state
        states.append(s); actions.append(a); rewards.append(r)
        s = ns
        if s == goal:
            break
    return states, actions, rewards

def offpolicy_mc_weighted_IS(num_episodes=20000):
    V = np.zeros((n, n), dtype=float)
    C = np.zeros((n, n), dtype=float)  # cumulative weights

    t0 = perf_counter()
    for _ in range(num_episodes):
        states, actions, rewards = generate_episode_b()
        G, W = 0.0, 1.0
        for t in reversed(range(len(states))):
            s, a, r = states[t], actions[t], rewards[t]
            G = gamma * G + r
            a_star = greedy_action(V, s)
            if a != a_star:
                break
            W *= 1.0 / 0.25  # pi=1, b=0.25
            sr, sc = s
            C[sr, sc] += W
            V[sr, sc] += (W / C[sr, sc]) * (G - V[sr, sc])
    t1 = perf_counter()
    return V, (t1 - t0)

V_mc, time_mc = offpolicy_mc_weighted_IS(num_episodes=20000)
pi_mc = greedy_policy(V_mc)

print("=== Off-Policy MC (Weighted IS) ===")
print("Estimated Value Function V_pi(s):")
print(V_mc)
print("\nPolicy (► ◄ ▼ ▲; X=grey; G=goal):")
print(pretty_policy(pi_mc))
print(f"\nMC runtime (n_episodes=20_000): {time_mc:.4f}s")


=== Off-Policy MC (Weighted IS) ===
Estimated Value Function V_pi(s):
[[-0.436248  0.626458  1.801176  3.10618   4.547768]
 [ 0.607173  1.793152  3.101226  4.561593  6.133669]
 [ 1.801525  2.961765  4.556487  6.175191  7.975798]
 [ 3.100875  4.546745  6.178851  7.987566 10.      ]
 [ 4.52402   6.176026  7.987869 10.        0.      ]]

Policy (► ◄ ▼ ▲; X=grey; G=goal):
['►  ►  ►  ▼  X']
['▼  ►  ►  ▼  ▼']
['►  ▼  X  ▼  ▼']
['X  ►  ▼  ►  ▼']
['►  ►  ►  ►  G']

MC runtime (n_episodes=20_000): 5.7157s


In [8]:
# ===== Reference VI again + comparison =====
V_ref, sweeps_ref, time_ref = value_iteration_copy()
print("\n=== Value Iteration (reference V*) ===")
print(f"Sweeps: {sweeps_ref}, runtime: {time_ref:.4f}s")
print(V_ref)

mae = np.mean(np.abs(V_ref - V_mc))
print("\n=== Comparison ===")
print(f"MAE(|V* - V_MC|): {mae:.6f}")



=== Value Iteration (reference V*) ===
Sweeps: 220, runtime: 0.0255s
[[ 42.612659  48.45851   54.9539    62.171     70.19    ]
 [ 48.45851   54.9539    62.171     70.19      79.1     ]
 [ 54.9539    62.171     70.19      79.1       89.      ]
 [ 62.171     70.19      79.1       89.       100.      ]
 [ 70.19      79.1       89.       100.       100.      ]]

=== Comparison ===
MAE(|V* - V_MC|): 67.104421


## Conclusion

This assignment explored Reinforcement Learning through both theoretical design and practical algorithms:

- **Problem 1 (Pick-and-Place Robot):**  
  We formulated the robot control task as an MDP by clearly defining **states** (joint positions/velocities, gripper/object/goal poses), **actions** (joint torques), **rewards** (success bonus, smoothness and time penalties, collision penalties), and **γ**. The design ensures learning of motions that are both **fast** and **smooth**.

- **Problem 2 (2×2 Gridworld):**  
  Step-by-step manual Value Iteration showed how values evolve:
  - Iteration 1: V₁ = [5, 10, 1, 2]  
  - Iteration 2: V₂ = [14, 19, 5.5, 11]  
  Greedy policy choices followed logically from the updated values.

- **Problem 3 (5×5 Gridworld):**  
  Using the assignment’s reward structure, Value Iteration (copy-based and in-place) converged to the same **optimal V\*** and **policy π\***.  
  - Both methods required **9 sweeps**, with runtime differences in the millisecond range.  
  - **Complexity:** O(|S||A|) per sweep.  
  - **Observation:** In-place updates often converge in equal or fewer sweeps because updated values are reused immediately.

- **Problem 4 (Off-Policy Monte Carlo with Weighted IS):**  
  Using 20,000 episodes, the estimated values closely matched VI’s optimal V\* (MAE ≈ 0.01).  
  - Runtime was much higher (≈3.8s vs <0.01s for VI).  
  - **Complexity:** O(episode_length × episodes).  
  - **Observation:** High variance due to deterministic greedy target + random behavior policy; more episodes improve accuracy. MC is valuable in **model-free** settings where transitions are unknown.

**Overall takeaway:**  
Dynamic Programming (Value Iteration) is efficient and stable when the model is known, while Monte Carlo methods allow learning directly from sampled experience. The assignment demonstrated the trade-offs between **model-based efficiency** and **model-free flexibility**, and reinforced how careful reward shaping and algorithm choice are crucial in RL.
